# Basic Mesh Generation with GMSHFlow

This notebook demonstrates the basic workflow for creating a triangular mesh using GMSHFlow.

We'll cover:
1. Creating a simple geometric domain
2. Setting up mesh parameters
3. Generating the mesh with GMSH
4. Exporting results

    "## Prerequisites\n",
    "- GMSH must be installed and available in your system PATH\n",
    "- Required packages: gmshflow, geopandas, shapely, matplotlib\n",
    "\n",
    "### Installation\n",
    "```bash\n",
    "# Install required packages\n",
    "pip install gmsh geopandas matplotlib\n",
    "\n",
    "# Install gmshflow (if not already installed)\n",
    "pip install -e .\n",
    "```"

In [ ]:
# Import required libraries
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt
import gmshflow

print(f"GMSHFlow version: {gmshflow.__version__}")

## Step 1: Create a Simple Domain

Let's start by creating a simple rectangular domain using Shapely and GeoPandas.

In [ ]:
# Create a simple rectangular domain
domain_coords = [
    (0, 0),
    (1000, 0),
    (1000, 500),
    (0, 500),
    (0, 0)
]

# Create polygon geometry
domain_polygon = Polygon(domain_coords)

# Create GeoDataFrame
domain_gdf = gpd.GeoDataFrame(
    {'name': ['domain']}, 
    geometry=[domain_polygon]
)

# Visualize the domain
fig, ax = plt.subplots(figsize=(8, 4))
domain_gdf.plot(ax=ax, alpha=0.7, edgecolor='black')
ax.set_title('Simple Rectangular Domain')
ax.set_xlabel('X coordinate (m)')
ax.set_ylabel('Y coordinate (m)')
ax.grid(True, alpha=0.3)
plt.show()

print(f"Domain bounds: {domain_gdf.bounds}")
print(f"Domain area: {domain_gdf.geometry.area.iloc[0]:.0f} m²")

## Step 2: Initialize GMSH Model and Domain

Now we'll set up the GMSH model and create a mesh domain with our geometry.

In [ ]:
# Define mesh parameters
mesh_size = 50.0  # Target element size in meters
model_name = "basic_example"

# Initialize GMSH model using context manager (recommended)
with gmshflow.GmshModel(model_name) as gmsh_model:
    print(f"GMSH model '{model_name}' initialized successfully")
    
    # Create mesh domain
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=mesh_size
    )
    
    print(f"Mesh domain created with target element size: {mesh_size} m")

## Step 3: Prepare the Mesh Domain

Before generating the mesh, we need to prepare the domain by setting up the geometry in GMSH.

In [ ]:
with gmshflow.GmshModel(model_name) as gmsh_model:
    # Create mesh domain
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=mesh_size
    )
    
    # Prepare the mesh domain (process geometry)
    mesh_domain.prepare_mesh_domain(
        mesh_area=1,      # Minimum area threshold for mesh elements
        min_overlap=0.1   # Minimum overlap threshold for geometry processing
    )
    print("Domain geometry prepared")
    
    # Create domain boundary loop
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    print(f"Domain boundary created with loop ID: {boundary_loop}")
    
    # Create the surface that will be meshed
    surface_id = mesh_domain.create_domain_surface()
    print(f"Mesh surface created with ID: {surface_id}")

## Step 4: Generate the Mesh

Now we can generate the triangular mesh using GMSH.

In [ ]:
with gmshflow.GmshModel(model_name) as gmsh_model:
    # Recreate the domain setup
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=mesh_size
    )
    
    mesh_domain.prepare_mesh_domain(mesh_area=1, min_overlap=0.1)
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    surface_id = mesh_domain.create_domain_surface()
    
    # Generate the mesh
    print("Generating mesh...")
    gmsh_model.generate_mesh()
    print("Mesh generation completed!")
    
    # Get mesh statistics
    import gmsh
    node_tags, node_coords, _ = gmsh.model.mesh.getNodes()
    element_types, element_tags, _ = gmsh.model.mesh.getElements()
    
    num_nodes = len(node_tags)
    num_elements = sum(len(tags) for tags in element_tags)
    
    print(f"Mesh statistics:")
    print(f"  - Number of nodes: {num_nodes}")
    print(f"  - Number of elements: {num_elements}")
    print(f"  - Target element size: {mesh_size} m")

## Step 5: Export to Voronoi Format

Export the triangular mesh to Voronoi format, which is useful for groundwater flow modeling.

In [ ]:
import os
from pathlib import Path

# Create output directory
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)

with gmshflow.GmshModel(model_name) as gmsh_model:
    # Recreate the domain setup
    mesh_domain = gmshflow.GmshMeshDomain(
        name="main_domain",
        gdf_dom=domain_gdf,
        cs_dom=mesh_size
    )
    
    mesh_domain.prepare_mesh_domain(mesh_area=1, min_overlap=0.1)
    boundary_loop = mesh_domain.create_domain_loop_from_poly()
    surface_id = mesh_domain.create_domain_surface()
    
    # Generate mesh
    gmsh_model.generate_mesh()
    
    # Export to Voronoi format
    output_name = "basic_mesh_voronoi"
    surface_tags = [surface_id]
    
    print("Exporting to Voronoi format...")
    voronoi_gdf = mesh_domain.export_to_voronoi(
        ws=output_dir,
        name=output_name,
        surface_ids=surface_tags
    )
    
    print(f"Voronoi mesh exported to: {output_dir / output_name}.shp")
    print(f"Number of Voronoi cells: {len(voronoi_gdf)}")

## Step 6: Visualize the Results

Let's visualize both the original domain and the generated Voronoi mesh.

In [ ]:
# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot original domain
domain_gdf.plot(ax=ax1, alpha=0.7, edgecolor='black', color='lightblue')
ax1.set_title('Original Domain')
ax1.set_xlabel('X coordinate (m)')
ax1.set_ylabel('Y coordinate (m)')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot Voronoi mesh
voronoi_gdf.plot(ax=ax2, alpha=0.7, edgecolor='red', linewidth=0.5, color='lightgreen')
ax2.set_title(f'Voronoi Mesh ({len(voronoi_gdf)} cells)')
ax2.set_xlabel('X coordinate (m)')
ax2.set_ylabel('Y coordinate (m)')
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

# Save the figure
fig.savefig(output_dir / 'basic_mesh_comparison.png', dpi=150, bbox_inches='tight')
print(f"Comparison plot saved to: {output_dir / 'basic_mesh_comparison.png'}")

## Summary

In this basic example, we've demonstrated:

1. **Domain Creation**: Created a simple rectangular domain using Shapely and GeoPandas
2. **GMSH Setup**: Initialized a GMSH model and mesh domain with target element size
3. **Mesh Preparation**: Prepared the domain geometry for meshing
4. **Mesh Generation**: Generated a triangular mesh using GMSH
5. **Export**: Exported the mesh to Voronoi format for use in groundwater modeling
6. **Visualization**: Created plots to visualize the results

### Key Takeaways:
- Always use the context manager (`with gmshflow.GmshModel()`) to ensure proper GMSH initialization and cleanup
- The `cell_size` parameter controls the target element size in the mesh
- The Voronoi export creates shapefiles that can be used with MODFLOW6 and other groundwater modeling software
- GMSHFlow handles the complex GMSH API calls, making mesh generation straightforward

### Next Steps:
- Try different `cell_size` values to see how they affect mesh resolution
- Experiment with more complex domain shapes
- Add observation points or boundary features (covered in other examples)
- Integrate with MODFLOW6 for groundwater flow modeling